Author or notebook: Arun Chakravorty

In [ ]:
#general packages
import math
import numpy as np
import pandas as pd
import glob
import json
#image processing packages
import tifffile as tf
from skimage.measure import regionprops
from PIL import Image
#plotting packages
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
def CreateDictionary(image, pos, positionfile):
    '''
    From a given ROI map, creates a dictionary of the locations
    
    Parameters
    ----------
    image: numpy array of mask
    pos: position number
    positionfile:dataframe of position information from .pos file
    '''
    #get information about masks
    regions = regionprops(image)
    finaldict = dict()
    for props in regions:
        y0, x0 = props.centroid
        # Now need to identify the proper offset
        subsetpositionfile = positionfile[positionfile['label'] == 'Pos' + str(int(pos))]
        x_final = x0 + subsetpositionfile['xpx'].values[0]
        y_final = y0 + subsetpositionfile['ypx'].values[0]
        fullkey = 'cell' + str(props.label) + '.0_pos' + str(pos)
        finaldict[fullkey] = [x0, y0, x_final, y_final, 'Pos'+str(int(pos))]
    return finaldict

def CreateDf(dictionary):
    tempdf = pd.DataFrame(dictionary)
    return tempdf

def Merge(dict1, dict2):
    return(dict2.update(dict1))

def get_stage_positions(filename, px_size=0.11):
    """
    getStagePositions:
    Parse a MicroManager position list to return the X,Y,Z
    positions and names of each position.
    
    Parameters
    ----------
    filename: position file src
    px_size: pixel size
    """

    with open(filename) as f:
        content = json.load(f)

    positions = []

    for pos in content['POSITIONS']:

        z_name = pos['DEFAULT_Z_STAGE']
        xy_name = pos['DEFAULT_XY_STAGE']
        
        gridcol = pos.get('GRID_COL', -1)
        gridrow = pos.get('GRID_ROW', -1)

        posinfo = {
            'label': pos['LABEL'],
            'gridrow': gridrow,
            'gridcol': gridcol
        }
        

        for dev in pos['DEVICES']:

            if dev['DEVICE'] == z_name:
                posinfo['z'] = dev['X']

            if dev['DEVICE'] == xy_name:
                posinfo['x'] = dev['X']
                posinfo['xpx'] = round(dev['X']/px_size)
                posinfo['y'] = dev['Y']
                posinfo['ypx'] = round(dev['Y']/px_size)

        positions.append(posinfo)

    return positions

### Import the masks 

In [ ]:
# Using tifffile to read in an image as a numpy array
mask1 = tf.imread('/groups/CaiLab/personal/Lex/raw/051222_150genes_E9.5/notebook_pyfiles/edges_deleted/MMStack_Pos2.ome.tif')
mask1.max()

In [ ]:
# Identifying regions
regions = regionprops(mask1)
len(regions)

In [ ]:
#look at mask
plt.imshow((mask1).astype(int), cmap="gray")
plt.show()

### Test

In [ ]:
#get centroid for each mask
sns.reset_orig()

fig, ax = plt.subplots()
ax.imshow(mask1, cmap=plt.cm.gray)

testdict = dict()
for props in regions:
    #print(props.label)
    y0, x0 = props.centroid
    testdict[props.label] = [x0, y0]
    ax.plot(x0, y0, '.g', markersize=2)

In [ ]:
testdict = CreateDf(testdict)
testdict.T

### Position offsets 

In [ ]:
#read in position file
pos_file = '/groups/CaiLab/personal/Lex/raw/051222_150genes_E9.5/section2.pos'

In [ ]:
#get information
positions = get_stage_positions(pos_file)
pos_df = pd.DataFrame(positions)

for i in pos_df.index.values:
    pos_df.loc[i,'label'] = 'Pos'+str(i)

In [ ]:
pos_df

### Defining Function

In [ ]:
#create dictionary 
newdict = dict()
for pos in range(0,98): 
    filename = f'/groups/CaiLab/personal/Lex/raw/051222_150genes_E9.5/notebook_pyfiles/edges_deleted/MMStack_Pos{pos}.ome.tif'
    image = np.flipud(tf.imread(filename))
    tempdict = CreateDictionary(image, pos, pos_df)
    newdict.update(tempdict)

In [ ]:
#transpose and rename columns
CellLocationsAndPositions = pd.DataFrame(newdict).T
CellLocationsAndPositions = CellLocationsAndPositions.rename(columns = {0: "X_rel", 1: "Y_rel", 2: "X_final", 3: "Y_final", 4: "Pos"})

In [ ]:
CellLocationsAndPositions

In [ ]:
sns.set(rc={'figure.figsize':(100,100)})
sns.scatterplot(data = CellLocationsAndPositions, x='X_final', y='Y_final')
plt.show()

In [ ]:
#CellLocationsAndPositions.to_csv('/groups/CaiLab/personal/Arun/raw/20220305_TestesAutomation/notebook_pyfiles/post_analysis_files/calculated_cell_locations.csv')